In [1]:
# Import Packages

import numpy as np
import pandas as pd
import networkx as nx
import os
import matplotlib.pyplot as plt
import itertools

In [2]:
# Load Data

decade = 1950
url = f"https://raw.githubusercontent.com/leon-marx/billnius/main/songs_cleaned_decade/songs_{decade}.csv"
data = pd.read_csv(url, sep=';')

In [31]:
# Get words, songs and edges

words = []
songs = []
edges = []

for songid in range(len(data['lyrics'])):
    title = data.iloc[songid, 2]
    title = title[:-4]
    songs.append(title)
    lyrics = data.iloc[songid, 3]
    for word in lyrics.split(" "):
        words.append(word)
        tupple = (title, word)
        edges.append(tupple)
        
words, w_counts = np.unique(words, return_counts=True)
words = [x for _, x in sorted(zip(w_counts, words))][::-1]
w_counts = sorted(w_counts)[::-1]

In [39]:
# Create Network

G = nx.Graph()

# add songs as nodes
G.add_nodes_from(songs, bipartite=0)

# add words as nodes
G.add_nodes_from([
        (word, {"size": w_counts[i]}) for i, word in enumerate(words)
    ], bipartite=1)

# add edges as nodes
G.add_edges_from(edges)

In [30]:
# Export Bipartite Network
nx.write_gexf(G, f'{decade}_bipartite.gexf')

In [48]:
from networkx.algorithms import bipartite

songs = {n for n, d in G.nodes(data=True) if d["bipartite"] == 0}
words = set(G) - songs

G_words = bipartite.projected_graph(G, words)

KeyboardInterrupt: 

In [45]:
from networkx.algorithms.community import louvain_communities, greedy_modularity_communities

In [ ]:
communitites = louvain_communities(G_words)